# Generate test data for plotting functionalities based on existing dandisets

## Import modules and define functions

In [1]:
# import modules
import hdmf._version
import matplotlib.pyplot as plt
import numpy as np
import platform
import pynwb
import seaborn as sns
import uuid

from datetime import datetime
from dateutil.tz import tzlocal
from nwbwidgets import nwb2widget
from pynwb import NWBHDF5IO, NWBFile, TimeSeries
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, Fluorescence

%matplotlib inline

In [2]:
# define functions
def load_nwb_file(filename):
    directory = '..//test_data//'
    read_io = NWBHDF5IO((directory + filename), 'r')
    nwbfile_in = read_io.read()

    return nwbfile_in

## Generate initial NWB file

In [3]:
# generate the main nwb file
start_time = datetime.now(tz=tzlocal())
create_date = datetime.now(tz=tzlocal())

unique_identifier = str(uuid.uuid4())

hdmf_ver = 'v%s'%hdmf._version.get_versions()['version']
info = 'An example NWB file created with pynwb v%s (hdmf %s), Python v%s'%(pynwb.__version__,hdmf_ver,platform.python_version())

nwbfile_out = pynwb.NWBFile(session_description='Example with various datatypes for testing plotting functionality',
                        identifier=unique_identifier,
                        session_start_time=start_time,
                        file_create_date=create_date,
                        notes=info,
                        experimenter='No one',
                        experiment_description='We used a python script to synthesize this data.',
                        institution='Institute X',
                        lab='No Lab.')

## Add example data modules
### Multi-dimensional time series data + two photon image series

In [4]:
# load test file 
time_series_multi_dim_fname = 'sub-661968859_ses-681698752_behavior+ophys.nwb'
nwbfile_in = load_nwb_file(time_series_multi_dim_fname)
proc_example = nwbfile_in.processing['brain_observatory_pipeline']

#create device, imaging plan, optical channel used
device = pynwb.file.Device('CAM2P_1')
nwbfile_out.add_device(device)
optical_channel = OpticalChannel('optchan', 'description', 480.)
imaging_plane = nwbfile_out.create_imaging_plane('imaging_plane_1', optical_channel,
                                             description='imaging_plane_1',
                                             device=device, excitation_lambda=910.,
                                             imaging_rate=30., indicator='GCaMP6f',
                                             location='PFC', reference_frame='home')

#make imaging series 
two_p_example = proc_example['max_project'].data[:,:,:]
image_series = TwoPhotonSeries(name='image_series', 
                               dimension=[2], 
                               data = two_p_example,                                 # TO DO - add synthetic data here
                               imaging_plane=imaging_plane,
                               starting_frame=[0], 
                               starting_time=0.0, rate=1.0,
                               unit='seconds')
nwbfile_out.add_acquisition(image_series)


# store image segmentation output
mod = nwbfile_out.create_processing_module('ophys', 'contains optical physiology processed data')
img_seg = ImageSegmentation()
mod.add(img_seg)
ps = img_seg.create_plane_segmentation('output from image plane', imaging_plane, 'my_planeseg', image_series)

#add image mask
mask_example = proc_example['ImageSegmentation']['PlaneSegmentation'].image_mask     # TO DO - add synthetic data here
ps.add_roi(image_mask=mask_example[0])
ps.add_roi(image_mask=mask_example[1])

#add fluorescence and roi response - TO DO - add synthetic data here 
data_example = proc_example['Fluorescence']['DfOverF'].data[:,:2]                    # TO DO - add synthetic data here
timestamps_example = proc_example['Fluorescence']['DfOverF'].timestamps[:]

fl = Fluorescence()
mod.add(fl)
rt_region = ps.create_roi_table_region('the first of two ROIs', region=[0,1])
rrs = fl.create_roi_response_series('DfOverF', data_example, rt_region, unit='lumens', timestamps=timestamps_example)

### Behavioral events time series + trials table

In [5]:
# behavioral events time series
behavior_events_markers_fname = 'sub-anm369962_ses-20170309.nwb'
nwbfile_in = load_nwb_file(behavior_events_markers_fname)

lick_data_example = nwbfile_in.acquisition['lick_times'].time_series['lick_left_times'].data[:] 
lick_timestamps_example = nwbfile_in.acquisition['lick_times'].time_series['lick_left_times'].timestamps[:]

ts = TimeSeries(name='lick_left_times',
                data=lick_data_example,                                              # TO DO - add synthetic data here
                timestamps=lick_timestamps_example,
                unit='a.u.')
lick_times = pynwb.behavior.BehavioralEvents(name='lick_times', time_series=ts)

nwbfile_out.add_acquisition(lick_times)

In [6]:
# add trials table
trials_example = nwbfile_in.trials[:]

for index, data in trials_example.iterrows():
    nwbfile_out.add_trial(start_time=data.start_time, stop_time=data.stop_time)      # TO DO - add synthetic data here

for col_name, col_data in trials_example.iteritems():
    if col_name not in nwbfile_out.trials.colnames:
        nwbfile_out.add_trial_column(name=col_name, description='',data=col_data.to_list())   

### Spatial series

In [7]:
# spatial series with x vs. y instead of data vs. timestamps
N = 2001
timestamps = np.linspace(0, N, N) #np.arange(N)
ss1 = pynwb.behavior.SpatialSeries(name='spatial_series_1D',
                                   reference_frame='Zero is origin..?',
                                   data=np.cos(timestamps/4),
                                   timestamps=timestamps,
                                   comments='A wave...',
                                   description='Description of this...')

nwbfile_out.add_acquisition(ss1)

x = np.cos(timestamps/4)
y = np.sin(timestamps/5)
ss2 = pynwb.behavior.SpatialSeries(name='spatial_series_2D',
                                   reference_frame='Zero is origin..?',
                                   data=np.array([x,y]).T,
                                   timestamps=timestamps,
                                   comments='A wave...',
                                   description='Description of this...')

nwbfile_out.add_acquisition(ss2)

### Stimulus image series data

In [8]:
#optical series
imaging_series_fname = 'sub-P10HMH_ses-20060901_ecephys+image.nwb'
nwbfile_in = load_nwb_file(imaging_series_fname)

optical_series_example = nwbfile_in.stimulus['StimulusPresentation'].data[:,:,:,:]   # TO DO - add synthetic data here
timestamps_example=nwbfile_in.stimulus['StimulusPresentation'].timestamps[:]

os = pynwb.image.OpticalSeries(name='StimulusPresentation',
                               data=optical_series_example,
                               timestamps=timestamps_example,
                               distance=0.5,
                               field_of_view=[0.2,0.3,0.5],
                               orientation='lower left',
                               unit='meters')
nwbfile_out.add_stimulus(os)

### Time stamps alignment

In [9]:
# time stamps with whole session vs. all aligned to one location
time_stamps_aligned_fname = '10_03_19-1.nwb'
nwbfile_in = load_nwb_file(time_stamps_aligned_fname)

# add stimulus
vcss_example_1 = nwbfile_in.stimulus['index_000'].data[:]                             #TO DO - add synthetic data here
vcss_start_1 = nwbfile_in.stimulus['index_000'].starting_time
vcss_example_2 = nwbfile_in.stimulus['index_001'].data[:]
vcss_start_2 = nwbfile_in.stimulus['index_001'].starting_time

# add acquisition
vcs_example_1 = nwbfile_in.acquisition['index_000'].data[:]
vcs_start_1 = nwbfile_in.acquisition['index_000'].starting_time
vcs_example_2 = nwbfile_in.acquisition['index_001'].data[:]
vcs_start_2 = nwbfile_in.acquisition['index_001'].starting_time

# add devices and electrodes
device = nwbfile_out.create_device(name='Heka EPC10')
elec = nwbfile_out.create_icephys_electrode(name="elec0",
                                        description='an intracellular electrode',
                                        device=device)

# add vclamp stimulus series
vcss1 = pynwb.icephys.VoltageClampStimulusSeries(name="vclamp_stim_series1",
                                        data=vcss_example_1,
                                        starting_time=vcss_start_1,
                                        rate=10000.0,
                                        electrode=elec,
                                        gain=1.)
vcss2 = pynwb.icephys.VoltageClampStimulusSeries(name="vclamp_stim_series2",
                                        data=vcss_example_2,
                                        starting_time=vcss_start_2,
                                        rate=10000.0,
                                        electrode=elec,
                                        gain=1.)
# add vclamp seriess
vcs1 = pynwb.icephys.VoltageClampSeries(name="vclamp_series1",
                                        data=vcs_example_1,
                                        starting_time=vcs_start_1,
                                        rate=10000.0,
                                        electrode=elec,
                                        gain=100000000.0)
vcs2 = pynwb.icephys.VoltageClampSeries(name="vclamp_series2",
                                        data=vcs_example_2,
                                        starting_time=vcs_start_2,
                                        rate=10000.0,
                                        electrode=elec,
                                        gain=100000000.0)

nwbfile_out.add_stimulus(vcss1)
nwbfile_out.add_stimulus(vcss2)
nwbfile_out.add_acquisition(vcs1)
nwbfile_out.add_acquisition(vcs2)

### Single units

In [10]:
#single unit raster plots
single_units_fname = 'sub-anm184389_ses-20130207_behavior+ecephys.nwb'
nwbfile_in = load_nwb_file(single_units_fname)

#add electrodes info
device = nwbfile_out.create_device(name='Neuronexus_32chan')
electrode_name = 'Neuronexus1'
description = "an example probe"
location = "somewhere in the hippocampus"
electrode_group = nwbfile_out.create_electrode_group(electrode_name,
                                                 description=description,
                                                 location=location,
                                                 device=device)

electrode_example = nwbfile_in.electrodes[:]
for idx, elec in electrode_example.iterrows():
    nwbfile_out.add_electrode(id=idx,                                                #TO DO - add synthetic data here
                          x=elec.x, y=elec.y, z=elec.z,
                          imp=elec.imp,
                          location='CA1', filtering='none',
                          group=electrode_group)

# add single units info
units_example = nwbfile_in.units[:]
for idx, unit in units_example.iterrows():
    nwbfile_out.add_unit(id=idx,                                                     #TO DO - add synthetic data here
                     spike_times=unit.spike_times,
                     waveform_mean=unit.waveform_mean,
                     waveform_sd = unit.waveform_sd)

### Save the data file

In [11]:
with NWBHDF5IO('..//test_data//test-plotting-datatypes.nwb', 'w') as export_io:
    export_io.write(nwbfile_out)

/Users/stephanieprince/opt/anaconda3/envs/gsoc-project/lib/python3.8/site-packages/hdmf/build/objectmapper.py:233: DtypeConversionWarning: Spec 'TimeSeries/timestamps': Value with data type float64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
/Users/stephanieprince/opt/anaconda3/envs/gsoc-project/lib/python3.8/site-packages/hdmf/build/objectmapper.py:233: DtypeConversionWarning: Spec 'RoiResponseSeries/timestamps': Value with data type float64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
/Users/stephanieprince/opt/anaconda3/envs/gsoc-project/lib/python3.8/site-packages/hdmf/build/objectmapper.py:233: DtypeConversionWarning: Spec 'OpticalSeries/timestamps': Value with data type float64 is being converted to data type float64 as specified.
  warnings.warn(full_warning_msg, DtypeConversionWarning)
/Users/stephanieprince/opt/anaconda3/envs/gsoc-project/lib/p

### Inspect the output file with NWB widget

In [12]:
nwbfile_out = load_nwb_file('..//test_data//test_plotting_datatypes.nwb')
nwb2widget(nwbfile_out)